In [ ]:
!python -m pip install pandas texstat

In [ ]:
import pandas as pd
import textstat

In [ ]:
def readability(essay):
    return textstat.flesch_reading_ease(essay)

def ttr(essay):
    words = essay.split()
    unique_words = set(words)
    return len(unique_words) / len(words)

In [ ]:
test_set = pd.read_csv("data/joint_test_dataset.csv")
test_set['Readability'] = test_set['EssayText'].apply(readability)
test_set['TTR'] = test_set['EssayText'].apply(ttr)
test_set